In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1hGVJJ0MERaP1ouEc_vcfFg1nS9aHeYw3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1hGVJJ0MERaP1ouEc_vcfFg1nS9aHeYw3" -O dataset-large.zip && rm -rf /tmp/cookies.txt

--2021-01-22 16:06:45--  https://docs.google.com/uc?export=download&confirm=1ydf&id=1hGVJJ0MERaP1ouEc_vcfFg1nS9aHeYw3
Resolving docs.google.com (docs.google.com)... 172.217.13.78, 2607:f8b0:4004:808::200e
Connecting to docs.google.com (docs.google.com)|172.217.13.78|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-5g-docs.googleusercontent.com/docs/securesc/q9kkr7t13qqib58jgp6rtnb2tfv4m6fc/7ssa0qf22k933mdfpqrl7rcaq3h9mt8c/1611331575000/04205927457588895310/09881555074476116646Z/1hGVJJ0MERaP1ouEc_vcfFg1nS9aHeYw3?e=download [following]
--2021-01-22 16:06:45--  https://doc-14-5g-docs.googleusercontent.com/docs/securesc/q9kkr7t13qqib58jgp6rtnb2tfv4m6fc/7ssa0qf22k933mdfpqrl7rcaq3h9mt8c/1611331575000/04205927457588895310/09881555074476116646Z/1hGVJJ0MERaP1ouEc_vcfFg1nS9aHeYw3?e=download
Resolving doc-14-5g-docs.googleusercontent.com (doc-14-5g-docs.googleusercontent.com)... 172.217.13.65, 2607:f8b0:4004:808::2001
Connecting to doc-14-5

In [ ]:
! mkdir dataset

In [ ]:
! unzip /content/dataset-large.zip -d /content/dataset/

In [ ]:
import tensorflow as tf
import time
from tensorflow import keras
import numpy as np
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Input, BatchNormalization, Dropout
from tensorflow.keras.layers import UpSampling2D, Concatenate, Dropout, Conv2DTranspose
import json
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.utils import Sequence

In [ ]:
! nvidia-smi

In [ ]:
class Unet:
    def __init__(self, input_shape):
        self.input_shape = input_shape
        self.model = tf.keras.Sequential()
        self.L = 0
        self.inputs = None

    def down_sampling_block(self, inputs, num_conv_block, num_filter, kernel_size, activation):
        for i in range(num_conv_block):
            if self.L == 0:
                inputs = Conv2D(filters=num_filter, kernel_size=kernel_size,
                                activation=activation, padding='same')(self.inputs)
            else:
                inputs = Conv2D(filters=num_filter, kernel_size=kernel_size,
                                activation=activation, padding='same')(inputs)
            self.L += 1
        outputs = MaxPool2D(pool_size=(2, 2))(inputs)
        self.L += 1
        return outputs, inputs

    def conv_block(self, inputs, num_conv_block, num_filter, kernel_size, activation):
        for i in range(num_conv_block):
            if self.L == 0:
                inputs = Conv2D(filters=num_filter, kernel_size=kernel_size,
                                activation=activation, padding='same')(self.inputs)
            else:
                inputs = (Conv2D(filters=num_filter, kernel_size=kernel_size,
                                 activation=activation, padding='same'))(inputs)
            self.L += 1
        outputs = inputs
        return outputs

    def up_sampling_block(self, inputs, concat, num_conv_block, num_filter, kernel_size, activation):
        for i in range(num_conv_block):
            if self.L == 0:
                inputs = Conv2D(filters=num_filter, kernel_size=kernel_size,
                                activation=activation, padding='same')(self.inputs)
            else:
                inputs = (Conv2D(filters=num_filter, kernel_size=kernel_size,
                                 activation=activation, padding='same'))(inputs)
            self.L += 1
        #outputs = UpSampling2D((2, 2))(inputs)[..., num_filter//2:]
        outputs = Conv2DTranspose(filters=num_filter//2, kernel_size=kernel_size,
                                  strides=(2, 2), padding='same')(inputs)
        shape_enc = concat.shape[1]
        shape_dec = outputs.shape[1]
        index = (shape_enc-shape_dec)//2
        concat = concat[:, index:index+shape_dec, index:index+shape_dec, :]
        outputs = Concatenate()([concat, outputs])
        return outputs

    def build(self):
        self.inputs = Input(shape=self.input_shape)
        down1, concat1 = self.down_sampling_block(inputs=None, num_conv_block=2, num_filter=32,
                                                  kernel_size=(3, 3), activation='relu')
        down2, concat2 = self.down_sampling_block(inputs=down1, num_conv_block=2, num_filter=64,
                                                  kernel_size=(3, 3), activation='relu')
        down3, concat3 = self.down_sampling_block(inputs=down2, num_conv_block=2, num_filter=128,
                                                  kernel_size=(3, 3), activation='relu')
        down4, concat4 = self.down_sampling_block(inputs=down3, num_conv_block=2, num_filter=256,
                                                  kernel_size=(3, 3), activation='relu')
        up1 = self.up_sampling_block(inputs=down4, concat=concat4, num_conv_block=2, num_filter=512,
                                     kernel_size=(3, 3), activation='relu')
        up2 = self.up_sampling_block(inputs=up1, concat=concat3, num_conv_block=2, num_filter=256,
                                     kernel_size=(3, 3), activation='relu')
        up3 = self.up_sampling_block(inputs=up2, concat=concat2, num_conv_block=2, num_filter=128,
                                     kernel_size=(3, 3), activation='relu')
        up4 = self.up_sampling_block(inputs=up3, concat=concat1, num_conv_block=2, num_filter=64,
                                     kernel_size=(3, 3), activation='relu')
        conv1 = self.conv_block(inputs=up4, num_conv_block=2, num_filter=32,
                                kernel_size=(3, 3), activation='relu')
        output = self.conv_block(inputs=conv1, num_conv_block=1, num_filter=1,
                                 kernel_size=(1, 1), activation='sigmoid')
        model = Model(self.inputs, output)
        return model

In [ ]:
class DataAugmentation:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size

    def resize_image(self, image, mask):
        w = image.shape[1]
        h = image.shape[0]
        ratio = min(w, h)/max(w, h)
        if w >= h:
            new_w = self.input_size
            new_h = round(self.input_size*ratio)
        else:
            new_h = self.input_size
            new_w = round(self.input_size*ratio)
        image_resized = cv2.resize(image, (new_w, new_h))
        mask_resized = cv2.resize(mask, (new_w, new_h))
        return image_resized, mask_resized

    def padding_image(self, image, mask):
        h = image.shape[0]
        w = image.shape[1]
        image_padded = cv2.copyMakeBorder(image,
                                          top=(self.input_size - h) // 2,
                                          bottom=self.input_size - (self.input_size - h) // 2 - h,
                                          left=(self.input_size - w) // 2,
                                          right=self.input_size - (self.input_size - w) // 2 - w,
                                          borderType=cv2.BORDER_CONSTANT)
        mask_padded = cv2.copyMakeBorder(mask,
                                         top=(self.input_size - h) // 2,
                                         bottom=self.input_size - (self.input_size - h) // 2 - h,
                                         left=(self.input_size - w) // 2,
                                         right=self.input_size - (self.input_size - w) // 2 - w,
                                         borderType=cv2.BORDER_CONSTANT)
        return image_padded, mask_padded

    @staticmethod
    def random_rotation(image, mask):
        degree = np.random.uniform(0, 360)
        h = image.shape[0]
        w = image.shape[1]
        rot_map = cv2.getRotationMatrix2D((w//2, h//2), degree, scale=1)
        image_rotated = cv2.warpAffine(image, rot_map, (w, h))
        mask_rotated = cv2.warpAffine(mask, rot_map, (w, h))
        return image_rotated, mask_rotated

    @staticmethod
    def random_flip(image, mask):
        if np.random.choice([0, 1]):
            flip_code = np.random.choice([-1, 0, 1])
            image_flipped = cv2.flip(image, flipCode=flip_code)
            mask_flipped = cv2.flip(mask, flipCode=flip_code)
            return image_flipped, mask_flipped
        return image, mask

    @staticmethod
    def random_blur(image, mask):
        if np.random.choice([0, 1]):
            radius = np.random.choice([1, 3, 5])
            image_blur = cv2.GaussianBlur(image, ksize=(radius, radius), sigmaX=1)
            mask_blur = cv2.GaussianBlur(mask, ksize=(radius, radius), sigmaX=1)
            return image_blur, mask_blur
        return image, mask

    @staticmethod
    def random_add_brightness(image):
        value = np.random.randint(0, 50)
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        lim = 255 - value
        v[v > lim] = 255
        v[v <= lim] += value

        image_brightness = cv2.merge((h, s, v))
        image_brightness = cv2.cvtColor(image_brightness, cv2.COLOR_HSV2BGR)
        return image_brightness

    def data_process(self, image, mask):
        image_processed, mask_processed = self.resize_image(image, mask)
        image_processed, mask_processed = self.padding_image(image_processed, mask_processed)
        image_processed, mask_processed = self.random_rotation(image_processed, mask_processed)
        image_processed, mask_processed = self.random_flip(image_processed, mask_processed)
        image_processed, mask_processed = self.random_blur(image_processed, mask_processed)
        image_processed = self.random_add_brightness(image_processed)
        image_processed = cv2.cvtColor(image_processed, cv2.COLOR_RGB2GRAY)
        mask_processed = cv2.resize(mask_processed, dsize=(self.output_size, self.output_size))
        (thresh, mask_processed) = cv2.threshold(mask_processed, 127, 255, cv2.THRESH_BINARY)
        image_processed = image_processed/255.0
        mask_processed = mask_processed/255.0
        return image_processed, mask_processed

In [ ]:
class DataLoader(Sequence):
    def __init__(self, meta_data_path, batch_size, phase='train', input_size=224, output_size=224):
        self.meta_data_path = meta_data_path
        self.batch_size = batch_size
        self.phase = phase
        self.input_size = input_size
        self.output_size = output_size
        if self.phase == 'train':
            self.abs_image_path = "/content/dataset/Original/Training/"
            self.abs_mask_path = "/content/dataset/MASKS/Training/"
        else:
            self.abs_image_path = "/content/dataset/Original/Testing/"
            self.abs_mask_path = "/content/dataset/MASKS/Testing/"
        self.train_path = None
        self.test_path = None
        self.indexes = None
        self.read_meta_data()

    def read_meta_data(self):
        files = open(self.meta_data_path)
        files = json.load(files)
        self.train_path = files['train'][:]
        self.test_path = files['test'][:]
        self.indexes = np.arange(len(self.train_path))
        self.indexes_test = np.arange(len(self.test_path))
        return self.train_path, self.test_path

    def process_image(self, image_paths):
        data_transform = DataAugmentation(input_size=self.input_size, output_size=self.output_size)
        x_train = []
        y_train = []
        for path in image_paths:
            image = cv2.imread(self.abs_image_path + path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # original image in jpg format but mask in png format
            mask = cv2.imread(self.abs_mask_path + path[:len(path) - 3] + "png", cv2.IMREAD_GRAYSCALE)
            image_processed, mask_processed = data_transform.data_process(image, mask)
            image_processed = np.expand_dims(image_processed, axis=2)
            mask_processed = np.expand_dims(mask_processed, axis=2)
            x_train.append(tf.convert_to_tensor(image_processed))
            y_train.append(tf.convert_to_tensor(mask_processed))
        return x_train, y_train

    def __getitem__(self, index):
        if self.phase == 'train':
            index_list = self.indexes
            data_path = self.train_path
        else:
            index_list = self.indexes_test
            data_path = self.test_path
        if index == self.__len__()-1:
            indexes = index_list[index * self.batch_size:]
        else:
            indexes = index_list[index*self.batch_size:(index+1)*self.batch_size]
        image_paths = [data_path[k] for k in indexes]
        image, mask = self.process_image(image_paths)
        image = tf.convert_to_tensor(image)
        mask = tf.convert_to_tensor(mask)
        return image, mask

    def __len__(self):
        if self.phase == 'train':
            num_path = len(self.train_path)
        else:
            num_path = len(self.test_path)
        return int(np.ceil(num_path / self.batch_size))

In [ ]:
META_DATA_PATH = "/content/dataset/data.json"
TRAINING_DATA_PATH = "/content/dataset/Original/Training/"
TRAINING_MASK_PATH = "/content/dataset/MASKS/Training/"
TESTING_DATA_PATH = "/content/dataset/Original/Testing/"
TESTING_MASK_PATH = "/content/dataset/MASKS/Testing/"

In [ ]:
# Hyper parameters
BATCH_SIZE = 32
LR = 0.0015
EPOCHS = 30
MOMENTUM = 0.9
NUM_CLASS = 2

In [ ]:
def scheduler(epoch, lr):
    if epoch != 0 and epoch % 5 == 0:
        return lr * 0.3
    else:
        return lr

In [ ]:
train_generator = DataLoader(META_DATA_PATH, batch_size=BATCH_SIZE, phase='train')
test_generator = DataLoader(META_DATA_PATH, batch_size=BATCH_SIZE, phase='test')

In [ ]:
model = Unet(input_shape=(224, 224, 1)).build()

In [ ]:
smooth = 1e-15
def iou(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    return (intersection + smooth) / (union + smooth)

In [ ]:
losses = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
#optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
#metric = tf.keras.metrics.MeanIoU(num_classes=2)

In [ ]:
model.compile(optimizer=optimizer, loss=losses, metrics=iou)

In [ ]:
model.summary()

In [ ]:
model.fit_generator(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=EPOCHS,
                    callbacks=[callback],
                    validation_data=test_generator,
                    validation_steps=len(test_generator))


In [ ]:
model.save('/content/drive/MyDrive/BAP Colab/HairSegmentation/model.hdf5')

In [ ]:
model.save_weights('/content/drive/MyDrive/BAP Colab/HairSegmentation/model_weight.h5')